In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint
import warnings

# Suppress warnings for cleaner output
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=FutureWarning)

# --- 1. Data Loading and Preprocessing ---
print("--- Starting Data Loading and Preprocessing ---")
try:
    df = pd.read_csv('dataset.csv')
    print("Dataset loaded successfully.")
except FileNotFoundError:
    print("Error: 'dataset.csv' not found. Please ensure the dataset file is in the correct directory.")
    exit() # Exit if the core dataset is not found

# Convert 'date' column to datetime objects
df['date'] = pd.to_datetime(df['date'])

# Handle missing values: Fill numerical NaNs with 0 (or a more appropriate strategy if known)
# For simplicity, filling with 0. A more robust approach might use mean/median or advanced imputation.
numerical_cols = df.select_dtypes(include=np.number).columns.tolist()
for col in numerical_cols:
    if df[col].isnull().any():
        df[col] = df[col].fillna(0) # Filling numerical NaNs with 0

print("Missing values handled.")

# Identify and remove rows with insufficient data for feature engineering (e.g., less than 30 days for 30-day volatility)
# Group by crypto_name and remove groups that are too small
min_rows_for_features = 30 # For 30-day volatility
df = df.groupby('crypto_name').filter(lambda x: len(x) >= min_rows_for_features).copy()
print(f"Removed cryptocurrencies with less than {min_rows_for_features} days of data.")

# --- 2. Feature Engineering ---
print("\n--- Starting Feature Engineering ---")

# Sort data for correct time-series calculations
df = df.sort_values(by=['crypto_name', 'date']).reset_index(drop=True)

# Calculate Daily Returns
df['daily_return'] = df.groupby('crypto_name')['close'].pct_change()
df['daily_return'] = df['daily_return'].replace([np.inf, -np.inf], np.nan).fillna(0) # Handle inf values

# Calculate Historical Volatility (Standard Deviation of Daily Returns)
# Using rolling window for 7, 14, and 30 days
df['volatility_7_day'] = df.groupby('crypto_name')['daily_return'].transform(lambda x: x.rolling(window=7).std())
df['volatility_14_day'] = df.groupby('crypto_name')['daily_return'].transform(lambda x: x.rolling(window=14).std())
df['volatility_30_day'] = df.groupby('crypto_name')['daily_return'].transform(lambda x: x.rolling(window=30).std())

# Simple Moving Averages (SMA) for Close Price
df['SMA_7'] = df.groupby('crypto_name')['close'].transform(lambda x: x.rolling(window=7).mean())
df['SMA_20'] = df.groupby('crypto_name')['close'].transform(lambda x: x.rolling(window=20).mean())

# Average True Range (ATR) - for volatility
# True Range (TR)
df['high-low'] = df['high'] - df['low']
df['high-prev_close'] = abs(df['high'] - df['close'].shift(1))
df['low-prev_close'] = abs(df['low'] - df['close'].shift(1))
df['TR'] = df[['high-low', 'high-prev_close', 'low-prev_close']].max(axis=1)
df['ATR_14'] = df.groupby('crypto_name')['TR'].transform(lambda x: x.rolling(window=14).mean())

# Volume to Market Cap Ratio
# Handle potential division by zero if marketCap is 0
df['volume_market_cap_ratio'] = df['volume'] / df['marketCap']
df['volume_market_cap_ratio'] = df['volume_market_cap_ratio'].replace([np.inf, -np.inf], 0) # Handle inf values
df['volume_market_cap_ratio'] = df['volume_market_cap_ratio'].fillna(0) # Handle NaN from division by zero or initial NaNs

# Drop auxiliary columns used for ATR calculation
df.drop(columns=['high-low', 'high-prev_close', 'low-prev_close', 'TR'], inplace=True)

# Drop rows with NaN values resulting from rolling window calculations (e.g., first few rows for each crypto)
df.dropna(inplace=True)

print("Feature engineering complete. Dataset shape after engineering:", df.shape)

# Normalize numerical features (excluding daily_return and volatility targets if not scaling them for direct prediction)
# We will scale OHLC, volume, marketCap, SMA, and ATR. Volatility targets are already in a range suitable for regression.
features_to_scale = ['open', 'high', 'low', 'close', 'volume', 'marketCap', 'SMA_7', 'SMA_20', 'ATR_14']
scaler = MinMaxScaler()
df[features_to_scale] = scaler.fit_transform(df[features_to_scale])
print("Numerical features normalized.")

# --- 3. Exploratory Data Analysis (EDA) ---
print("\n--- Starting Exploratory Data Analysis (EDA) ---")

# Summary Statistics
print("\n--- Summary Statistics of Numerical Features ---")
print(df.describe().to_markdown(numalign="left", stralign="left"))

# Cryptocurrency Distribution
print("\n--- Number of Records per Cryptocurrency ---")
print(df['crypto_name'].value_counts().to_markdown(numalign="left", stralign="left"))

# Get the top 5 cryptocurrencies by number of records for plotting
top_cryptos = df['crypto_name'].value_counts().head(5).index.tolist()

# Time-Series Trends (Close Price and 30-day Volatility) for Top Cryptos
print("\n--- Plotting Time-Series Trends for Top Cryptocurrencies ---")
plt.figure(figsize=(18, 10))
for i, crypto in enumerate(top_cryptos):
    crypto_df = df[df['crypto_name'] == crypto].sort_values(by='date')

    # Plot Close Price
    plt.subplot(2, len(top_cryptos), i + 1)
    plt.plot(crypto_df['date'], crypto_df['close'], label='Close Price', color='skyblue')
    plt.title(f'{crypto} Close Price Trend')
    plt.xlabel('Date')
    plt.ylabel('Normalized Close Price')
    plt.grid(True)
    plt.tight_layout()

    # Plot 30-day Volatility
    plt.subplot(2, len(top_cryptos), i + 1 + len(top_cryptos))
    plt.plot(crypto_df['date'], crypto_df['volatility_30_day'], label='30-Day Volatility', color='salmon')
    plt.title(f'{crypto} 30-Day Volatility Trend')
    plt.xlabel('Date')
    plt.ylabel('30-Day Volatility')
    plt.grid(True)
    plt.tight_layout()

plt.suptitle('Time-Series Trends for Top Cryptocurrencies (Normalized Price and Volatility)', y=1.02, fontsize=16)
plt.show()

# Distribution of Volatility Features
print("\n--- Plotting Distribution of Volatility Features ---")
plt.figure(figsize=(15, 5))

plt.subplot(1, 3, 1)
sns.histplot(df['volatility_7_day'], kde=True, bins=50, color='blue')
plt.title('Distribution of 7-Day Volatility')
plt.xlabel('Volatility')
plt.ylabel('Frequency')

plt.subplot(1, 3, 2)
sns.histplot(df['volatility_14_day'], kde=True, bins=50, color='green')
plt.title('Distribution of 14-Day Volatility')
plt.xlabel('Volatility')
plt.ylabel('Frequency')

plt.subplot(1, 3, 3)
sns.histplot(df['volatility_30_day'], kde=True, bins=50, color='red')
plt.title('Distribution of 30-Day Volatility')
plt.xlabel('Volatility')
plt.ylabel('Frequency')

plt.tight_layout()
plt.suptitle('Distribution of Different Volatility Measures', y=1.02, fontsize=16)
plt.show()

# Correlation Analysis
print("\n--- Correlation Matrix ---")
correlation_cols = ['open', 'high', 'low', 'close', 'volume', 'marketCap',
                    'daily_return', 'volatility_7_day', 'volatility_14_day', 'volatility_30_day',
                    'SMA_7', 'SMA_20', 'ATR_14', 'volume_market_cap_ratio']
corr_matrix = df[correlation_cols].corr()

plt.figure(figsize=(14, 10))
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', fmt=".2f", linewidths=.5)
plt.title('Correlation Matrix of Features and Volatility')
plt.show()
print("EDA complete. Visualizations displayed.")


# --- 4. Model Training (RandomForestRegressor) ---
print("\n--- Starting Model Training ---")

# Define features (X) and target (y)
features = ['open', 'high', 'low', 'close', 'volume', 'marketCap',
            'daily_return', 'SMA_7', 'SMA_20', 'ATR_14', 'volume_market_cap_ratio']
target = 'volatility_30_day'

# Drop rows where target or features might be NaN after feature engineering (e.g., initial rolling window NaNs)
df.dropna(subset=features + [target], inplace=True)

X = df[features]
y = df[target]

# Chronological split for time-series data (80% training, 20% testing)
split_date_index = int(len(df) * 0.8)
split_date = df['date'].iloc[split_date_index]

train_df = df[df['date'] < split_date].copy()
test_df = df[df['date'] >= split_date].copy()

X_train = train_df[features]
y_train = train_df[target]
X_test = test_df[features]
y_test = test_df[target]

print(f"Splitting data chronologically at date: {split_date}")
print(f"Training set size: {len(X_train)} samples")
print(f"Testing set size: {len(X_test)} samples")

# Initialize and train the RandomForestRegressor model (initial model)
model = RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1)
print("\nTraining the initial RandomForestRegressor...")
model.fit(X_train, y_train)
print("Initial Model training complete.")


# --- 5. Model Evaluation (Initial Model) ---
print("\n--- Initial Model Evaluation ---")

# Make predictions on the test set
y_pred = model.predict(X_test)

# Calculate evaluation metrics
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Root Mean Squared Error (RMSE): {rmse:.4f}")
print(f"Mean Absolute Error (MAE): {mae:.4f}")
print(f"R-squared (R2) Score: {r2:.4f}")

# Visualize Actual vs. Predicted Volatility (Subset for clarity)
plt.figure(figsize=(15, 6))
plt.plot(y_test.values[::50], label='Actual Volatility (30-day)', alpha=0.7)
plt.plot(y_pred[::50], label='Predicted Volatility (30-day)', alpha=0.7)
plt.title('Actual vs. Predicted 30-Day Volatility (Initial Model - Subset of Test Data)')
plt.xlabel('Sample Index (every 50th point)')
plt.ylabel('30-Day Volatility')
plt.legend()
plt.grid(True)
plt.show()
print("Initial model evaluation complete. Visualizations displayed.")


# --- 6. Hyperparameter Tuning ---
print("\n--- Starting Hyperparameter Tuning (RandomizedSearchCV) ---")

# Define the parameter distribution for RandomizedSearchCV
param_dist = {
    'n_estimators': randint(50, 200),
    'max_features': ['sqrt', 'log2'],
    'max_depth': randint(10, 50),
    'min_samples_split': randint(2, 10),
    'min_samples_leaf': randint(1, 5)
}

# Initialize RandomizedSearchCV (n_jobs=1 to avoid pickling errors)
random_search = RandomizedSearchCV(
    estimator=RandomForestRegressor(random_state=42),
    param_distributions=param_dist,
    n_iter=10,  # Number of parameter settings that are sampled. Can increase for more thorough search.
    cv=3,
    verbose=0, # Set to 0 to suppress verbose output for cleaner script
    random_state=42,
    n_jobs=1, # Crucial for avoiding pickling errors in some environments
    scoring='neg_mean_squared_error'
)

random_search.fit(X_train, y_train)

print("\n--- Hyperparameter Tuning Complete ---")
print(f"Best parameters found: {random_search.best_params_}")
print(f"Best cross-validation score (negative RMSE): {-random_search.best_score_:.4f}")

# Get the best model from the search
best_model = random_search.best_estimator_


# --- 7. Re-evaluation of Tuned Model ---
print("\n--- Re-evaluating the Model with Best Parameters ---")

y_pred_tuned = best_model.predict(X_test)

rmse_tuned = np.sqrt(mean_squared_error(y_test, y_pred_tuned))
mae_tuned = mean_absolute_error(y_test, y_pred_tuned)
r2_tuned = r2_score(y_test, y_pred_tuned)

print(f"RMSE (Tuned Model): {rmse_tuned:.4f}")
print(f"MAE (Tuned Model): {mae_tuned:.4f}")
print(f"R-squared (R2) Score (Tuned Model): {r2_tuned:.4f}")

# Visualize Actual vs. Predicted Volatility (Scatter Plot) for Tuned Model
plt.figure(figsize=(12, 6))
plt.scatter(y_test, y_pred_tuned, alpha=0.3)
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', lw=2)
plt.xlabel('Actual Volatility (30-day)')
plt.ylabel('Predicted Volatility (30-day)')
plt.title('Actual vs. Predicted 30-day Volatility (Tuned Model)')
plt.grid(True)
plt.show()

# Visualize predictions over time for a specific cryptocurrency from the test set (Tuned Model)
if not test_df['crypto_name'].empty:
    crypto_for_viz = test_df['crypto_name'].value_counts().index[0]
    crypto_test_df_viz = test_df[test_df['crypto_name'] == crypto_for_viz].sort_values(by='date').copy()
    
    crypto_test_df_viz['predicted_volatility'] = best_model.predict(crypto_test_df_viz[features])

    plt.figure(figsize=(15, 7))
    plt.plot(crypto_test_df_viz['date'], crypto_test_df_viz['volatility_30_day'], label='Actual 30-Day Volatility', color='blue')
    plt.plot(crypto_test_df_viz['date'], crypto_test_df_viz['predicted_volatility'], label='Predicted 30-Day Volatility', color='red', linestyle='--')
    plt.title(f'Actual vs. Predicted 30-Day Volatility for {crypto_for_viz} (Tuned Model Test Set)')
    plt.xlabel('Date')
    plt.ylabel('Volatility')
    plt.legend()
    plt.grid(True)
    plt.show()
else:
    print("No cryptocurrencies found in the test set for time-series visualization.")

print("\n--- Source Code Execution Complete ---")
print("This script encompasses Data Processing, Feature Engineering, EDA, Model Training, Evaluation, and Tuning.")